# Klett inversion and Angstrom exponent retrieval

This code helps you to retrieve and plot lidar products. If you are not confident in Python, don't worry! You only need to make modifications to the statements found as follows:

#------------------------------------------------------------------------------------------------------------------------------

         Modify ONLY within here
#------------------------------------------------------------------------------------------------------------------------------

Important:
- A recurring error may be that paths are specified with '\\' as the folder separator, while the Python language uses '\\\\' (double).
- It is also recommended to run the cells one by one, and not to re-run a cell while it is executing (the software may hang). If while a cell is running, any changes are made to the code in that cell and we want to re-run it, we should wait for it to finish or restart the kernels.
- If desired, you can play with the rest of the code.

## IMPORTING LIBRARIES AND SPECIFYING PATHS

First, we need to import the necessary libraries. In case any of the modules are not found when running, install the module using:

#------------------------------------------------------------------------------------------------------------------------------

    !pip install <module_name>
#------------------------------------------------------------------------------------------------------------------------------


In [ ]:
from pathlib import Path

import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy import integrate
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.retrieval.klett import klett_rcs
from gfatpy.atmo.ecmwf import get_ecmwf_temperature_pressure
from gfatpy.atmo.freudenthaler_molecular_properties import molecular_properties
from gfatpy.lidar.utils import refill_overlap, signal_to_rcs
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.utils.plot import font_axes

%load_ext autoreload
%autoreload 2

%matplotlib inline

In the next cell, we define the directory where the lidar measurement files are located and the directory where we want to save the graphs.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
lidar_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\data\lidar')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
lidar_dir = Path(r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\02\22')
output_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

lidar_dir.is_dir(), output_dir.is_dir() #Comprueba que ambas carpetas existen.


## Read lidar data


In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
RS_FL = lidar_dir / 'alh_1a_Prs_rs_xf_20230222.nc' #lidar file
range_tuple = (0, 10000.0) #Range range to be used
#------------------------------------------------------------------------------------------------------------------------------

#Load and preprocess and range selection 
whole_lidar = preprocess(RS_FL, crop_ranges=range_tuple)


**KEEP IN MIND**: Do not change the range dimension of the lidar dataset from here

## ECMWF data and optical molecular properties
ECMWF data will be used to retrieve the optical molecular properties such as:
1. Molecular backscatter coefficient
1. Molecular extinction coefficient
1. Molecular lidar ratio

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
date_str = '2023-02-22' #lidar file
hour = 12
range_array = whole_lidar.range #ECMWF data will be interpolated to lidar range
#------------------------------------------------------------------------------------------------------------------------------

#Read ECMWF data
atmo = get_ecmwf_temperature_pressure(date_str, hour=hour, heights=range_array)
atmo

## Optical molecular properties
Retrieve molecular properties at wavelengths decided by the user

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
channels = ['1064fta', '532fta', '355fpa', '355npa', '532npa']
#------------------------------------------------------------------------------------------------------------------------------

#Retrieve molecular properties at elastic wavelengths
mol_properties = {}
for channel_ in channels:
    # Wavelength
    wavelength = int(channel_[:-3])
        
    mol_properties[channel_] = molecular_properties(
        wavelength, atmo["pressure"].values, atmo["temperature"].values, heights=range_array
    )

mol_properties['1064fta']

In [ ]:
#Save whole lidar dataset
quicklook_xarray(whole_lidar['signal_1064fta'], is_rcs=False, scale_bounds=(0,2e8))
plt.ylim(0,8000)


## Define a time period to average signals

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
time_tuple = ('2023-02-22 11:00:00', '2023-02-22 12:00:00') #Time range to be used
#------------------------------------------------------------------------------------------------------------------------------

#Define profile time to be averaged.
lidar = whole_lidar.sel(time=slice(*time_tuple))

#Check chosen period 
quicklook_xarray(lidar['signal_1064fta'], is_rcs=False, scale_bounds=(0,2e8))
plt.ylim(0,8000)


## Check range-correced signals 

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
normalization_range_tuple = (6000,7000) #Time range to be used
#------------------------------------------------------------------------------------------------------------------------------

fig, ax = plt.subplots(figsize=[8,5])

for channel_ in channels:   
    # Average the Range-corrected signal at  in the time range defined by time_tuple [commonly 30 min]
    rcs = signal_to_rcs(lidar[f'signal_{channel_}'].sel(time=slice(*time_tuple)).mean('time'), lidar.range)
    nrcs = rcs/rcs.sel(range=slice(*normalization_range_tuple)).mean('range')
    nrcs.plot(label=channel_) # type: ignore
plt.yscale('linear')
# plt.xlim(0,10000)
# #Plotting beta [part]
# ax1 = plt.subplot(1,4,1)
# ax1.plot(beta_total['355xta'], height, c='b', label='355part')
# ax1.plot(beta_total['532xpa'], height, c='green', label='532part')
# ax1.plot(beta_total['1064xta'], height, c='red', label='1064part')
# ax1.set_xlabel('$\\beta_{part}, [m^{-1} \cdot sr^{-1}]$')
plt.ylabel('RCS, [a.u.]')
plt.xlabel('Range, [m]')
# ax1.set_xlim(-5e-7,1.25e-5)
# ax1.set_ylim([0,8000])
plt.legend(fontsize='large')

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
reference_height = (8000, 9000)
# channels = KEEP THE SAME AS BEFORE
lr_part = {'1064fta': 50, '532fta': 50, '355fpa': 50, '532npa': 50, '355npa': 50} 
beta_aer_ref = 0 
#------------------------------------------------------------------------------------------------------------------------------

#Define a dictionary to save the aerosol-particle backscattering coefficients by channels
beta_part = {}
beta_mol = {}
for channel_ in channels:   
    # Average the Range-corrected signal at  in the time range defined by time_tuple [commonly 30 min]
    rcs = signal_to_rcs(lidar[f'signal_{channel_}'].mean('time'), lidar.range)
    
    # Wavelength
    wavelength = channel_[:-3]
        
    #Beta mol
    beta_mol[channel_] = mol_properties[channel_]['molecular_beta']
    
    # Retrieve aerosol-particle backscattering coefficient
    beta_part[channel_] = klett_rcs(rcs.values, 
                                      rcs.range.values, 
                                      beta_mol[channel_],
                                      lr_aer = lr_part[channel_],                                       
                                      reference = reference_height,
                                      beta_aer_ref = beta_aer_ref
                                     )

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
fulloverlap_height = 600
#------------------------------------------------------------------------------------------------------------------------------

#Define a dictionary to save the aerosol-particle extinction coefficients by channels
alpha_part ={}
aod = {}
od_total = {}
beta_total = {}
att_beta_total = {}

for channel_ in channels:       
    # Wavelength
    wavelength = int(channel_[:-3])

    # Retrieve extinction profile
    alpha_part[channel_] = beta_part[channel_]*lr_part[channel_]
    
    # Retrieve AOD
        #Set to zero the part of the extinction profile below the overlap height
    alpha_part[channel_] = refill_overlap(alpha_part[channel_], lidar.range.values, fulloverlap_height=fulloverlap_height)
        
        #Retrieve resolution
    resolution = np.median(np.diff(lidar.range.values)).item()

        #Retrieve AOD
    aod[channel_] = integrate.cumtrapz(alpha_part[channel_], dx=resolution, initial=0)

        #Retrieve OD
    alpha_mol = mol_properties[channel_]['molecular_alpha']
    od_total[channel_] = integrate.cumtrapz((alpha_part[channel_]+alpha_mol), dx=resolution, initial=0)

    # Retrieve total backscattering coefficient (mol + part)
    beta_total[channel_] = beta_part[channel_]+beta_mol[channel_]
    
    # Retrieve attenuated total backscattering coefficient 
    att_beta_total[channel_] = beta_total[channel_]*np.exp(-2*od_total[channel_])
    

## Angstrom exponent retrieval

Angstrom exponent provides the spectral dependence of the aerosol-particle backscattering coefficient as follows:
\begin{equation}
AE[\lambda_1, \lambda_2] = - \frac{ln \left(\frac{\beta_1}{\beta_2} \right)}{ln \left(\frac{\lambda_1}{\lambda_2} \right)}
\end{equation}

In [ ]:
# Channels
for channel_ in channels:
    beta_part[channel_] = refill_overlap(beta_part[channel_], lidar.range.values, fulloverlap_height=fulloverlap_height)

# Retrieval Agnstrom exponent
ae355_532 = -np.log(beta_part['532npa']/beta_part['355npa'])/np.log(532/355)
ae532_1064 = -np.log(beta_part['1064fta']/beta_part['532fta'])/np.log(1064/532)
ae355_1064 = -np.log(beta_part['1064fta']/beta_part['355fpa'])/np.log(1064/355)

#Retrieve AE for beta larger than a threshold
backscattering_ratio_532 = beta_total['532fta'] / beta_mol['532fta']
ae355_532 [backscattering_ratio_532<1.2] = np.nan
ae532_1064 [backscattering_ratio_532<1.2] = np.nan
ae355_1064 [backscattering_ratio_532<1.2] = np.nan

## Plot profiles

In [ ]:
height = lidar.range


fig, ax = plt.subplots(figsize=[15,10])
#Plotting beta [part]
ax1 = plt.subplot(1,3,1)
ax1.plot(beta_part['355fpa'], height, c='b', label='355part')
ax1.plot(beta_part['355npa'], height, c='lightblue', label='355part-nf')
ax1.plot(beta_part['532fta'], height, c='green', label='532part')
ax1.plot(beta_part['532npa'], height, c='green', label='532part-nf')
ax1.plot(beta_part['1064fta'], height, c='red', label='1064part')
ax1.set_xlabel('$\\beta_{part}, [m^{-1} \cdot sr^{-1}]$')
ax1.set_ylabel('Range, [m agl]')
ax1.set_xlim(-5e-7,5e-6)
ax1.set_ylim(0,8000)
ax1.legend(fontsize='small', loc='upper right')

#Plotting beta [mol and total]
ax2 = plt.subplot(1,3,2)
ax2.plot(beta_total['355fpa'], height, c='b', label='355total')
ax2.plot(beta_total['355npa'], height, c='violet', label='355total-nf')
ax2.plot(beta_mol['355fpa'], height, c='lightblue', lw=3, label='355mol')
ax2.plot(beta_total['532fta'], height, c='green', label='532total')
ax2.plot(beta_mol['532fta'], height, c='lightgreen', lw=3, label='532mol')
ax2.plot(beta_total['1064fta'], height, c='red', label='1064total')
ax2.plot(beta_mol['1064fta'], height, c='pink', lw=3, label='1064mol')
ax2.set_xlabel('$\\beta_{aer}, [m^{-1}\cdot sr^{-1}]$')
ax2.set_xlim(-5e-7,1.25e-5)
ax2.set_ylim(0,8000)
ax2.set_yticklabels([])
ax2.legend(fontsize='small', loc='upper right')

#Plotting AE
ax3 = plt.subplot(1,3,3)
ax3.plot(ae355_532, height, c='r', label='355-532 nm - nf')
ax3.plot(ae532_1064, height, c='b', label='532-1064 nm')
ax3.plot(ae355_1064, height, c='g', label='355-1064 nm')
ax3.set_xlim(-0.5,4)
ax3.set_xlabel('$Angs. exp., [\#]$')
ax3.legend(fontsize='small', loc='upper right')
ax3.set_ylim(0,8000)
ax3.set_yticklabels([])

# #Plotting attenuated backscattering coefficients
# ax4 = plt.subplot(1,4,4)
# ax4.plot(att_beta_total['355xtp'], height, c='b', label='355')
# ax4.plot(att_beta_total['532xpa'], height, c='g', label='532')
# ax4.plot(att_beta_total['1064xta'], height, c='r', label='1064')
# # ax3.set_xlim(-0.5,4)
# ax4.legend(fontsize='small', loc='upper right')
# ax4.set_xlabel('$\\beta_{att}, [m^{-1}\cdot sr^{-1}]$')
# ax4.set_ylim([0,8000])
# ax4.set_yticklabels([])
# for ax_ in [ax1, ax2, ax3, ax4]:
#     font_axes(ax_)